# Enrichment app data preparation

In [1]:
import pandas as pd
import csv

In [19]:
rename = {
    "Accession": "UniProtID",
    "FragID": "FragID",
    "Abundance_Ratio_Adj_PValue": "pAdj",
    "Abundance_Ratio_log2": "Log2FC",
    "Abundance_Ratio_log2_median_corr": "Log2FCMedian",
    "Abundance_Ratio_PValue": "p",
    "Rank_relative": "RankRelative",
    "Number_of_Protein_Unique_Peptides": "UniqPeptides",
    "exp_id": "expID"	
}

df = pd.read_csv("../data/general/screening.tsv", sep="\t")
df = df[[k for k in rename.keys()]]
df = df.rename(columns = rename)
df.to_csv("../data/general/cemm_primary_data.tsv", sep="\t", index=False)
print(df.shape)
df = df[df["UniqPeptides"] >= 2]
df = df[df["p"].notnull()]
df.to_csv("../data/general/cemm_primary_detected_data.tsv", sep="\t", index=False)
print(df.shape)

df = pd.read_csv("../data/general/screening_hits.tsv", sep="\t")
df = df[[k for k in rename.keys()]]
df = df.rename(columns = rename)
df.to_csv("../data/general/cemm_primary_hit_data.tsv", sep="\t", index=False)
print(df.shape)


(1357294, 9)
(1311295, 9)
(45988, 9)


In [47]:
df = pd.read_csv("../data/general/cemm_primary_data.tsv", sep="\t")
prots = set(df["UniProtID"])
with open("../data/general/pid2name_primary_all.tsv", "r") as f:
    with open("../data/general/pid2name_primary.tsv", "w") as g:
        reader = csv.reader(f, delimiter="\t")
        writer = csv.writer(g, delimiter="\t")
        for r in reader:
            if r[0] in prots:
                writer.writerow([r[0], r[1]])

To get code to generate protein annotations files, please refer to [this notebook](https://github.com/ligand-discovery/exploratory-analyses-miscellanea/blob/main/notebooks/09_ProteinEnrichment.ipynb).

In [23]:
dd = pd.read_csv("../data/general/cemm_primary_detected_data.tsv", delimiter="\t")
de = pd.read_csv("../data/general/cemm_primary_hit_data.tsv", delimiter="\t")

In [24]:
from networkx.algorithms import bipartite
import networkx as nx

B = nx.Graph()
B.add_nodes_from(sorted(set(de["FragID"])), bipartite=0)
B.add_nodes_from(sorted(set(de["UniProtID"])), bipartite=1)

pairs = []
for r in de[["FragID", "UniProtID"]].values:
    pairs += [(r[0], r[1])]

B.add_edges_from(pairs)

In [25]:
fids_e = sorted(set(de["FragID"]))
frag_betweenness_centrality = bipartite.centrality.betweenness_centrality(B, nodes=fids_e)
frag_closeness_centrality = bipartite.centrality.closeness_centrality(B, nodes=fids_e)
frag_degree_centrality = bipartite.centrality.degree_centrality(B, nodes=fids_e)

In [26]:
pids_e = sorted(set(de["UniProtID"]))
prot_betweenness_centrality = bipartite.centrality.betweenness_centrality(B, nodes=pids_e)
prot_closeness_centrality = bipartite.centrality.closeness_centrality(B, nodes=pids_e)
prot_degree_centrality = bipartite.centrality.degree_centrality(B, nodes=pids_e)

In [32]:
import numpy as np
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

R = []
for pid in pids_e:
    R += [[prot_degree_centrality[pid], prot_betweenness_centrality[pid], prot_closeness_centrality[pid]]]

trf = PowerTransformer()
#trf = QuantileTransformer(output_distribution="normal")
X = trf.fit_transform(R)
X = np.mean(X, axis=1)
R = []
for x in X:
    R += [[x]]
trf = QuantileTransformer(output_distribution="normal")
X = trf.fit_transform(R).ravel()

In [33]:
idxs = np.argsort(-X)

promiscuity = []
for i in idxs:
    promiscuity += [[pids_e[i], X[i]]]

with open("../data/signatures/proteins/global/promiscuity/promiscuity_gauss.tsv", "w") as f:
    writer = csv.writer(f, delimiter="\t")
    for r in promiscuity:
        writer.writerow(r)

In [34]:
for t in [50, 100, 250, 500]:
    R = promiscuity[:t]
    with open("../data/signatures/proteins/global/promiscuity/promiscuity_top_{0}.tsv".format(t), "w") as f:
        writer = csv.writer(f, delimiter="\t")
        for r in R:
            writer.writerow([r[0]])

for t in [50, 100, 250, 500]:
    R = promiscuity[-t:]
    with open("../data/signatures/proteins/global/promiscuity/promiscuity_bottom_{0}.tsv".format(t), "w") as f:
        writer = csv.writer(f, delimiter="\t")
        for r in R:
            writer.writerow([r[0]])


In [35]:
import collections
import numpy as np
from sklearn.preprocessing import QuantileTransformer

dd_counts = collections.defaultdict(int)
dd_upeps = collections.defaultdict(int)
for r in dd[["UniProtID", "UniqPeptides"]].values:
    dd_counts[r[0]] += 1
    dd_upeps[r[0]] += r[1]

R = []
pids_d = []
for k,v in dd_counts.items():
    R += [[v, dd_upeps[k]]]
    pids_d += [k]

trf = QuantileTransformer(output_distribution="normal")
X = trf.fit_transform(R)
X = np.mean(X, axis=1)
R = []
for x in X:
    R += [[x]]
trf = QuantileTransformer(output_distribution="normal")
X = trf.fit_transform(R).ravel()

In [36]:
idxs = np.argsort(-X)

detectability = []
for i in idxs:
    detectability += [[pids_d[i], X[i]]]

with open("../data/signatures/proteins/global/detectability/detectability_gauss.tsv", "w") as f:
    writer = csv.writer(f, delimiter="\t")
    for r in detectability:
        writer.writerow(r)

for t in [50, 100, 250, 500]:
    R = detectability[:t]
    with open("../data/signatures/proteins/global/detectability/detectability_top_{0}.tsv".format(t), "w") as f:
        writer = csv.writer(f, delimiter="\t")
        for r in R:
            writer.writerow([r[0]])

for t in [50, 100, 250, 500]:
    R = detectability[-t:]
    with open("../data/signatures/proteins/global/detectability/detectability_bottom_{0}.tsv".format(t), "w") as f:
        writer = csv.writer(f, delimiter="\t")
        for r in R:
            writer.writerow([r[0]])

In [37]:
det = {}
with open("../data/signatures/proteins/global/detectability/detectability_gauss.tsv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for r in reader:
        det[r[0]] = float(r[1])

prom = {}
with open("../data/signatures/proteins/global/promiscuity/promiscuity_gauss.tsv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for r in reader:
        prom[r[0]] = float(r[1])

In [38]:
frag2values = collections.defaultdict(list)
for r in dd[["FragID", "UniProtID", "Log2FC"]].values:
    frag2values[r[0]] += [(r[1], r[2])]

frag2values = dict((k, sorted(v, key=lambda x: -x[1])) for k, v in frag2values.items())

frag2hits = collections.defaultdict(set)
for r in de[["FragID", "UniProtID"]].values:
    frag2hits[r[0]].update([r[1]])

In [41]:
from tqdm import tqdm
import numpy as np

prot2ranks = collections.defaultdict(list)
for k, v in tqdm(frag2values.items()):
    for i, x in enumerate(v):
        prot2ranks[x[0]] += [i]

prot2ranks_mean = dict((k, np.mean(v)) for k, v in prot2ranks.items())
prot2ranks_std = dict((k, np.std(v)) for k, v in prot2ranks.items())

100%|██████████| 412/412 [00:00<00:00, 1536.62it/s]


In [43]:
import os
import shutil

for k, v in frag2values.items():
    dirname = os.path.join("../data/signatures/proteins/fragment/", k)
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
    with open(os.path.join(dirname, "{0}_val_log2fc.tsv".format(k)), "w") as f:
        writer = csv.writer(f, delimiter="\t")
        for x in v:
            writer.writerow(x)
    #with open(os.path.join(dirname, "{0}_val_zrank.tsv".format(k)), "w") as f:
    #    writer = csv.writer(f, delimiter="\t")
    #    for i, x in enumerate(v):
    #        z = (prot2ranks_mean[x[0]] - i) / prot2ranks_std[x[0]]
    #        writer.writerow([x[0], z])
    for top in [50, 100, 250, 500]:
        v_ = v[:top]
        with open(os.path.join(dirname, "{0}_bin_{1}.tsv".format(k, top)), "w") as f:
            writer = csv.writer(f, delimiter="\t")
            for x in v_:
                writer.writerow([x[0]])
    hits = frag2hits[k]
    hits = [x[0] for x in v if x[0] in hits]
    with open(os.path.join(dirname, "{0}_bin_hit.tsv".format(k)), "w") as f:
        writer = csv.writer(f, delimiter="\t")
        for x in hits:
            writer.writerow([x])

In [44]:
# Global properties of proteins

# Detectability
dirname = os.path.join("../data/signatures/proteins/global/detectability")
if os.path.exists(dirname):
    shutil.rmtree(dirname)
os.mkdir(dirname)
d = collections.defaultdict(int)
for p in list(dd["UniProtID"]):
    d[p] += 1
v = sorted(d.items(), key=lambda x: -x[1])
with open(os.path.join(dirname, "detectability_val_counts.tsv"), "w") as f:
    writer = csv.writer(f, delimiter="\t")
    for x in v:
        writer.writerow(x)
for top in [1500, 3000]:
    with open(os.path.join(dirname, "detectability_bin_{0}.tsv".format(top)), "w") as f:
        v_ = v[:top]
        writer = csv.writer(f, delimiter="\t")
        for x in v_:
            writer.writerow([x[0]])

# Number of hits
dirname = os.path.join("../data/signatures/proteins/global/promiscuity_hit")
if os.path.exists(dirname):
    shutil.rmtree(dirname)
os.mkdir(dirname)
d = collections.defaultdict(int)
for p in list(de["UniProtID"]):
    d[p] += 1
v = sorted(d.items(), key=lambda x: -x[1])
with open(os.path.join(dirname, "promiscuity_hit_val_counts.tsv"), "w") as f:
    writer = csv.writer(f, delimiter="\t")
    for x in v:
        writer.writerow(x)
for top in [50, 100, 250, 500]:
    with open(
        os.path.join(dirname, "promiscuity_hit_bin_{0}.tsv".format(top)), "w"
    ) as f:
        v_ = v[:top]
        writer = csv.writer(f, delimiter="\t")
        for x in v_:
            writer.writerow([x[0]])

In [45]:
# tfidf_specificity

from sklearn.feature_extraction.text import TfidfTransformer

dirname = os.path.join("../data/signatures/proteins/global/tfidf_specificity")
if os.path.exists(dirname):
    shutil.rmtree(dirname)
os.mkdir(dirname)
pairs = []
for v in de[["UniProtID", "FragID"]].values:
    pairs += [(v[0], v[1])]
pairs = set(pairs)

pids = sorted(set([p[0] for p in pairs]))
fids = sorted(set([p[1] for p in pairs]))
X = np.zeros((len(pids), len(fids)))
for i, pid in enumerate(pids):
    for j, fid in enumerate(fids):
        if (pid, fid) in pairs:
            X[i, j] += 1

transf = TfidfTransformer()
Tp = np.array(transf.fit_transform(X).todense())
spec = 1 / np.sum(Tp, axis=1)

d = {}
for p, s in zip(pids, spec):
    d[p] = s
d = sorted(d.items(), key=lambda x: -x[1])

with open(os.path.join(dirname, "tfidf_specificity_val_by_sum.tsv"), "w") as f:
    writer = csv.writer(f, delimiter="\t")
    for r in d:
        writer.writerow(r)

transf = TfidfTransformer()
Tf = np.array(transf.fit_transform(X.T).todense())
fspec = 1 / np.sum(Tf, axis=1)

d = {}
for i, pid in enumerate(pids):
    mask = X[i, :] > 0
    d[pid] = np.mean(fspec[mask])
d = sorted(d.items(), key=lambda x: -x[1])

with open(os.path.join(dirname, "tfidf_specificity_val_by_mean_frag.tsv"), "w") as f:
    writer = csv.writer(f, delimiter="\t")
    for r in d:
        writer.writerow(r)